# Run FlowSOM for pixel and cell clustering

In [1]:
%load_ext autoreload
%autoreload 2

import harpy as hp
from harpy.datasets import pixie_example
from harpy.table.cell_clustering._utils import _export_to_ark_format as _export_to_ark_format_cells
from harpy.table.pixel_clustering._cluster_intensity import _export_to_ark_format as _export_to_ark_format_pixels
from harpy.utils._keys import ClusteringKey

/Users/arnedf/miniconda3/envs/harpy/lib/python3.10/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(


## Load example dataset

In [2]:
sdata_ark_analysis = pixie_example(["fov0", "fov1"])
sdata_ark_analysis

/Users/arnedf/miniconda3/envs/harpy/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-11 11:14:28,429 - harpy.image._manager - WARNING - No dims parameter specified. Assuming order of dimension of provided array is ((c), (z), y, x)
2024-12-11 11:14:28,551 - harpy.image._manager - INFO - Writing results to layer 'raw_image_fov0'
2024-12-11 11:14:28,555 - harpy.image._manager - WARNING - No dims parameter specified. Assuming order of dimension of provided array is ((c), (z), y, x)
2024-12-11 11:14:28,560 - harpy.image._manager - INFO - Writing results to layer 'label_nuclear_fov0'
2024-12-11 11:14:28,563 - harpy.image._manager - WARNING - No dims parameter specified. Assuming order of dimension of provided array is ((c), (z), y, x)
2024-12-11 11:14:28,567 - harpy.image._manager - INFO - Writing resu

/Users/arnedf/.cache/huggingface/datasets/downloads/extracted/ed276a09a07145a5c25cd3c0a3fd99368fc2f3387300f55927c0b600c043de39/post_clustering


2024-12-11 11:14:28,623 - harpy.image._manager - WARNING - No dims parameter specified. Assuming order of dimension of provided array is ((c), (z), y, x)
2024-12-11 11:14:28,717 - harpy.image._manager - INFO - Writing results to layer 'raw_image_fov1'
2024-12-11 11:14:28,721 - harpy.image._manager - WARNING - No dims parameter specified. Assuming order of dimension of provided array is ((c), (z), y, x)
2024-12-11 11:14:28,727 - harpy.image._manager - INFO - Writing results to layer 'label_nuclear_fov1'
2024-12-11 11:14:28,730 - harpy.image._manager - WARNING - No dims parameter specified. Assuming order of dimension of provided array is ((c), (z), y, x)
2024-12-11 11:14:28,735 - harpy.image._manager - INFO - Writing results to layer 'label_whole_fov1'
/Users/arnedf/miniconda3/envs/harpy/lib/python3.10/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/arn

SpatialData object
├── Images
│     ├── 'raw_image_fov0': DataArray[cyx] (22, 512, 512)
│     └── 'raw_image_fov1': DataArray[cyx] (22, 1024, 1024)
├── Labels
│     ├── 'label_nuclear_fov0': DataArray[yx] (512, 512)
│     ├── 'label_nuclear_fov1': DataArray[yx] (1024, 1024)
│     ├── 'label_whole_fov0': DataArray[yx] (512, 512)
│     └── 'label_whole_fov1': DataArray[yx] (1024, 1024)
└── Tables
      └── 'table': AnnData (1414, 22)
with coordinate systems:
    ▸ 'fov0', with elements:
        raw_image_fov0 (Images), label_nuclear_fov0 (Labels), label_whole_fov0 (Labels)
    ▸ 'fov1', with elements:
        raw_image_fov1 (Images), label_nuclear_fov1 (Labels), label_whole_fov1 (Labels)

In [3]:
channels = [
    "CD3",
    "CD4",
    "CD8",
    "CD14",
    "CD20",
    "CD31",
    "CD45",
    "CD68",
    "CD163",
    "CK17",
    "Collagen1",
    "Fibronectin",
    "ECAD",
    "HLADR",
    "SMA",
    "Vim",
]

In [4]:
sdata_ark_analysis = hp.im.pixel_clustering_preprocess(
    sdata_ark_analysis,
    img_layer=["raw_image_fov0", "raw_image_fov1"],
    output_layer=["raw_image_fov0_processed", "raw_image_fov1_processed"],
    channels=channels,
    chunks=2048,
    overwrite=True,
    sigma=2.0,
)
sdata_ark_analysis

2024-12-11 11:14:29,159 - harpy.image._manager - WARNING - No dims parameter specified. Assuming order of dimension of provided array is ((c), (z), y, x)
2024-12-11 11:14:29,929 - harpy.image._manager - INFO - Writing results to layer 'raw_image_fov0_processed'
2024-12-11 11:14:29,931 - harpy.image._manager - WARNING - No dims parameter specified. Assuming order of dimension of provided array is ((c), (z), y, x)
2024-12-11 11:14:30,716 - harpy.image._manager - INFO - Writing results to layer 'raw_image_fov1_processed'


SpatialData object
├── Images
│     ├── 'raw_image_fov0': DataArray[cyx] (22, 512, 512)
│     ├── 'raw_image_fov0_processed': DataArray[cyx] (16, 512, 512)
│     ├── 'raw_image_fov1': DataArray[cyx] (22, 1024, 1024)
│     └── 'raw_image_fov1_processed': DataArray[cyx] (16, 1024, 1024)
├── Labels
│     ├── 'label_nuclear_fov0': DataArray[yx] (512, 512)
│     ├── 'label_nuclear_fov1': DataArray[yx] (1024, 1024)
│     ├── 'label_whole_fov0': DataArray[yx] (512, 512)
│     └── 'label_whole_fov1': DataArray[yx] (1024, 1024)
└── Tables
      └── 'table': AnnData (1414, 22)
with coordinate systems:
    ▸ 'fov0', with elements:
        raw_image_fov0 (Images), raw_image_fov0_processed (Images), label_nuclear_fov0 (Labels), label_whole_fov0 (Labels)
    ▸ 'fov1', with elements:
        raw_image_fov1 (Images), raw_image_fov1_processed (Images), label_nuclear_fov1 (Labels), label_whole_fov1 (Labels)

In [5]:
import flowsom as fs
from dask.distributed import Client, LocalCluster

work_with_client = False

if work_with_client:
    # client example
    cluster = LocalCluster(
        n_workers=1,
        threads_per_worker=10,
    )

    client = Client(cluster)
else:
    client = None

batch_model=fs.models.BatchFlowSOMEstimator

sdata_ark_analysis, fsom, mapping = hp.im.flowsom(
    sdata_ark_analysis,
    img_layer=[ "raw_image_fov0_processed", "raw_image_fov1_processed" ],
    output_layer_clusters=[
        "raw_image_fov0_flowsom_clusters",
        "raw_image_fov1_flowsom_clusters",
    ],  # we need output_cluster_layer and output_meta_cluster_layer --> these will both be labels layers
    output_layer_metaclusters=[
        "raw_image_fov0_flowsom_metaclusters",
        "raw_image_fov1_flowsom_metaclusters",
        ],
    n_clusters=20,
    random_state=111,
    chunks=512,
    client = client,
    model = batch_model,
    num_batches = 10,
    xdim=10,
    ydim=10,
    persist_intermediate=True,
    overwrite=True,
)
sdata_ark_analysis

2024-12-11 11:14:30.894 | DEBUG    | flowsom.main:__init__:84 - Reading input.
2024-12-11 11:14:30.895 | DEBUG    | flowsom.main:__init__:86 - Fitting model: clustering and metaclustering.
2024-12-11 11:14:32.906 | DEBUG    | flowsom.main:__init__:88 - Updating derived values.
2024-12-11 11:14:33,626 - harpy.image._manager - WARNING - No dims parameter specified. Assuming order of dimension of provided array is ((c), (z), y, x)
2024-12-11 11:14:33,629 - harpy.image._manager - INFO - Writing results to layer 'raw_image_fov0_flowsom_clusters'
2024-12-11 11:14:33,630 - harpy.image._manager - WARNING - No dims parameter specified. Assuming order of dimension of provided array is ((c), (z), y, x)
2024-12-11 11:14:33,633 - harpy.image._manager - INFO - Writing results to layer 'raw_image_fov0_flowsom_metaclusters'
2024-12-11 11:14:33,951 - harpy.image._manager - WARNING - No dims parameter specified. Assuming order of dimension of provided array is ((c), (z), y, x)
2024-12-11 11:14:33,957 - 

SpatialData object
├── Images
│     ├── 'raw_image_fov0': DataArray[cyx] (22, 512, 512)
│     ├── 'raw_image_fov0_processed': DataArray[cyx] (16, 512, 512)
│     ├── 'raw_image_fov1': DataArray[cyx] (22, 1024, 1024)
│     └── 'raw_image_fov1_processed': DataArray[cyx] (16, 1024, 1024)
├── Labels
│     ├── 'label_nuclear_fov0': DataArray[yx] (512, 512)
│     ├── 'label_nuclear_fov1': DataArray[yx] (1024, 1024)
│     ├── 'label_whole_fov0': DataArray[yx] (512, 512)
│     ├── 'label_whole_fov1': DataArray[yx] (1024, 1024)
│     ├── 'raw_image_fov0_flowsom_clusters': DataArray[yx] (512, 512)
│     ├── 'raw_image_fov0_flowsom_metaclusters': DataArray[yx] (512, 512)
│     ├── 'raw_image_fov1_flowsom_clusters': DataArray[yx] (1024, 1024)
│     └── 'raw_image_fov1_flowsom_metaclusters': DataArray[yx] (1024, 1024)
└── Tables
      └── 'table': AnnData (1414, 22)
with coordinate systems:
    ▸ 'fov0', with elements:
        raw_image_fov0 (Images), raw_image_fov0_processed (Images), label_nuclea

In [6]:
sdata_ark_analysis = hp.tb.cluster_intensity(
    sdata_ark_analysis,
    mapping=mapping,
    img_layer=["raw_image_fov0_processed", "raw_image_fov1_processed"],
    labels_layer=["raw_image_fov0_flowsom_clusters", "raw_image_fov1_flowsom_clusters"],
    to_coordinate_system=[ "fov0", "fov1" ],
    output_layer="counts_clusters",
    overwrite=True,
)
sdata_ark_analysis

/Users/arnedf/VIB/harpy/src/harpy/table/_allocation_intensity.py:217: ImplicitModificationWarning: Setting element `.obsm['spatial']` of view, initializing view as actual.
  adata.obsm[_SPATIAL] = coordinates
/Users/arnedf/VIB/harpy/src/harpy/table/_allocation_intensity.py:217: ImplicitModificationWarning: Setting element `.obsm['spatial']` of view, initializing view as actual.
  adata.obsm[_SPATIAL] = coordinates
/Users/arnedf/miniconda3/envs/harpy/lib/python3.10/site-packages/spatialdata/_core/_elements.py:116: UserWarning: Key `counts_clusters` already exists. Overwriting it in-memory.
  self._check_key(key, self.keys(), self._shared_keys)
2024-12-11 11:14:35,924 - harpy.table._preprocess - INFO - Calculating cell size from provided labels_layer 'raw_image_fov0_flowsom_clusters'
2024-12-11 11:14:35,941 - harpy.table._preprocess - INFO - Calculating cell size from provided labels_layer 'raw_image_fov1_flowsom_clusters'
/Users/arnedf/miniconda3/envs/harpy/lib/python3.10/site-packages/

SpatialData object
├── Images
│     ├── 'raw_image_fov0': DataArray[cyx] (22, 512, 512)
│     ├── 'raw_image_fov0_processed': DataArray[cyx] (16, 512, 512)
│     ├── 'raw_image_fov1': DataArray[cyx] (22, 1024, 1024)
│     └── 'raw_image_fov1_processed': DataArray[cyx] (16, 1024, 1024)
├── Labels
│     ├── 'label_nuclear_fov0': DataArray[yx] (512, 512)
│     ├── 'label_nuclear_fov1': DataArray[yx] (1024, 1024)
│     ├── 'label_whole_fov0': DataArray[yx] (512, 512)
│     ├── 'label_whole_fov1': DataArray[yx] (1024, 1024)
│     ├── 'raw_image_fov0_flowsom_clusters': DataArray[yx] (512, 512)
│     ├── 'raw_image_fov0_flowsom_metaclusters': DataArray[yx] (512, 512)
│     ├── 'raw_image_fov1_flowsom_clusters': DataArray[yx] (1024, 1024)
│     └── 'raw_image_fov1_flowsom_metaclusters': DataArray[yx] (1024, 1024)
└── Tables
      ├── 'counts_clusters': AnnData (100, 16)
      └── 'table': AnnData (1414, 22)
with coordinate systems:
    ▸ 'fov0', with elements:
        raw_image_fov0 (Images), 

In [7]:
batch_model=fs.models.BatchFlowSOMEstimator

sdata_ark_analysis, fsom = hp.tb.flowsom(
    sdata_ark_analysis,
    labels_layer_cells=["label_whole_fov0", "label_whole_fov1"],
    labels_layer_clusters=[
        "raw_image_fov0_flowsom_metaclusters",
        "raw_image_fov1_flowsom_metaclusters",
    ],  # here you could also choose "ark_pixel_som_cluster"
    output_layer="table_cell_clustering_flowsom",
    chunks=512,
    model = batch_model,
    num_batches = 10,
    random_state=100,
    overwrite=True,
)
sdata_ark_analysis

2024-12-11 11:14:36,153 - harpy.table._preprocess - INFO - Calculating cell size from provided labels_layer 'label_whole_fov0'
2024-12-11 11:14:36,170 - harpy.table._preprocess - INFO - Calculating cell size from provided labels_layer 'label_whole_fov1'
/Users/arnedf/miniconda3/envs/harpy/lib/python3.10/site-packages/spatialdata/models/models.py:1048: UserWarning: Converting `region_key: fov_labels` to categorical dtype.
  return convert_region_column_to_categorical(adata)
/Users/arnedf/miniconda3/envs/harpy/lib/python3.10/site-packages/spatialdata/_core/_elements.py:116: UserWarning: Key `table_cell_clustering_flowsom` already exists. Overwriting it in-memory.
  self._check_key(key, self.keys(), self._shared_keys)
2024-12-11 11:14:36.219 | DEBUG    | flowsom.main:__init__:84 - Reading input.
2024-12-11 11:14:36.220 | DEBUG    | flowsom.main:__init__:86 - Fitting model: clustering and metaclustering.
2024-12-11 11:14:36.227 | DEBUG    | flowsom.main:__init__:88 - Updating derived value

SpatialData object
├── Images
│     ├── 'raw_image_fov0': DataArray[cyx] (22, 512, 512)
│     ├── 'raw_image_fov0_processed': DataArray[cyx] (16, 512, 512)
│     ├── 'raw_image_fov1': DataArray[cyx] (22, 1024, 1024)
│     └── 'raw_image_fov1_processed': DataArray[cyx] (16, 1024, 1024)
├── Labels
│     ├── 'label_nuclear_fov0': DataArray[yx] (512, 512)
│     ├── 'label_nuclear_fov1': DataArray[yx] (1024, 1024)
│     ├── 'label_whole_fov0': DataArray[yx] (512, 512)
│     ├── 'label_whole_fov1': DataArray[yx] (1024, 1024)
│     ├── 'raw_image_fov0_flowsom_clusters': DataArray[yx] (512, 512)
│     ├── 'raw_image_fov0_flowsom_metaclusters': DataArray[yx] (512, 512)
│     ├── 'raw_image_fov1_flowsom_clusters': DataArray[yx] (1024, 1024)
│     └── 'raw_image_fov1_flowsom_metaclusters': DataArray[yx] (1024, 1024)
└── Tables
      ├── 'counts_clusters': AnnData (100, 16)
      ├── 'table': AnnData (1414, 22)
      └── 'table_cell_clustering_flowsom': AnnData (1409, 20)
with coordinate systems:


In [8]:
# weighted channel average for visualization -> calculate this on the flowsom clustered matrix
sdata_ark_analysis = hp.tb.weighted_channel_expression(
    sdata_ark_analysis,
    table_layer_cell_clustering="table_cell_clustering_flowsom",
    table_layer_pixel_cluster_intensity="counts_clusters",
    output_layer="table_cell_clustering_flowsom",
    clustering_key=ClusteringKey._METACLUSTERING_KEY,
    overwrite=True,
)
sdata_ark_analysis

2024-12-11 11:14:36,436 - harpy.table.cell_clustering._weighted_channel_expression - INFO - Adding mean over obtained cell clusters '(clustering)' of the average marker expression for each cell weighted by pixel cluster count to '.uns[ 'clustering_channels' ]' of table layer 'table_cell_clustering_flowsom'
2024-12-11 11:14:36,437 - harpy.table.cell_clustering._weighted_channel_expression - INFO - Adding mean over obtained cell clusters '(metaclustering)' of the average marker expression for each cell weighted by pixel cluster count to '.uns[ 'metaclustering_channels' ]' of table layer 'table_cell_clustering_flowsom'
2024-12-11 11:14:36,438 - harpy.table.cell_clustering._weighted_channel_expression - INFO - Adding average marker expression for each cell weighted by pixel cluster count to '.obs' of table layer 'table_cell_clustering_flowsom'
/Users/arnedf/miniconda3/envs/harpy/lib/python3.10/site-packages/spatialdata/_core/_elements.py:116: UserWarning: Key `table_cell_clustering_flowsom

SpatialData object
├── Images
│     ├── 'raw_image_fov0': DataArray[cyx] (22, 512, 512)
│     ├── 'raw_image_fov0_processed': DataArray[cyx] (16, 512, 512)
│     ├── 'raw_image_fov1': DataArray[cyx] (22, 1024, 1024)
│     └── 'raw_image_fov1_processed': DataArray[cyx] (16, 1024, 1024)
├── Labels
│     ├── 'label_nuclear_fov0': DataArray[yx] (512, 512)
│     ├── 'label_nuclear_fov1': DataArray[yx] (1024, 1024)
│     ├── 'label_whole_fov0': DataArray[yx] (512, 512)
│     ├── 'label_whole_fov1': DataArray[yx] (1024, 1024)
│     ├── 'raw_image_fov0_flowsom_clusters': DataArray[yx] (512, 512)
│     ├── 'raw_image_fov0_flowsom_metaclusters': DataArray[yx] (512, 512)
│     ├── 'raw_image_fov1_flowsom_clusters': DataArray[yx] (1024, 1024)
│     └── 'raw_image_fov1_flowsom_metaclusters': DataArray[yx] (1024, 1024)
└── Tables
      ├── 'counts_clusters': AnnData (100, 16)
      ├── 'table': AnnData (1414, 22)
      └── 'table_cell_clustering_flowsom': AnnData (1409, 20)
with coordinate systems:


In [9]:
df = _export_to_ark_format_pixels(adata=sdata_ark_analysis["counts_clusters"], output=None)
(
    df_cell_som_cluster_count_avg,
    df_cell_som_cluster_channel_avg,
    df_cell_meta_cluster_channel_avg,
) = _export_to_ark_format_cells(sdata_ark_analysis, table_layer="table_cell_clustering_flowsom", output=None)
df

2024-12-11 11:14:36,543 - harpy.table.cell_clustering._utils - WARNING - Increasing cell cluster IDs (SOM cluster and meta cluster IDs) with +1 for visualization. The underlying dataframe in the SpatialData object remains unchanges.
2024-12-11 11:14:36,547 - harpy.table.cell_clustering._utils - WARNING - Increasing cell cluster IDs (SOM cluster and meta cluster IDs) with +1 for visualization. The underlying dataframe in the SpatialData object remains unchanges.
2024-12-11 11:14:36,550 - harpy.table.cell_clustering._utils - WARNING - Increasing cell cluster IDs (SOM cluster and meta cluster IDs) with +1 for visualization. The underlying dataframe in the SpatialData object remains unchanges.


channels,CD3,CD4,CD8,CD14,CD20,CD31,CD45,CD68,CD163,CK17,Collagen1,Fibronectin,ECAD,HLADR,SMA,Vim,pixel_meta_cluster,pixel_som_cluster,count
cells,,,,,,,,,,,,,,,,,,,
1_counts_clusters_a34405d3,79.662205,55.991234,3.339938,4.108338,3.976343,0.578644,40.246395,1.907112,2.754490,1.733413,7.401993,4.566187,2.153363,2.462132,1.229484,5.993419,5,1,11752
2_counts_clusters_a34405d3,44.357776,78.521997,1.993199,6.019438,2.279956,0.575271,29.573188,2.653170,4.149742,1.820865,9.875389,6.779230,2.466204,3.465180,1.839119,9.828461,5,2,9440
3_counts_clusters_a34405d3,60.862157,17.323038,7.604455,6.990412,2.871016,1.422096,21.476851,3.547263,6.289704,3.325128,19.664543,12.088449,7.148716,3.925060,5.011668,12.711450,4,3,7020
4_counts_clusters_a34405d3,13.080275,11.272689,5.681579,7.350961,7.338359,2.766095,79.544518,3.125768,6.229959,3.226597,10.637938,11.225132,3.957128,6.422467,2.363766,12.759784,17,4,7375
5_counts_clusters_a34405d3,49.938381,10.921948,49.406746,4.959634,5.208744,1.200178,53.361240,1.872839,3.809860,2.536913,5.482853,5.099018,2.056258,2.575106,1.056300,8.767088,2,5,11025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96_counts_clusters_a34405d3,3.981039,13.323010,3.211220,41.065997,1.524707,1.301879,12.672745,6.005695,9.931868,1.654690,34.965053,12.188016,2.678179,11.858054,6.403222,9.091599,9,96,15057
97_counts_clusters_a34405d3,2.245529,9.532662,2.118796,62.286725,0.782066,0.652548,9.386354,3.824479,8.670170,1.040886,41.990669,11.365051,1.014963,7.734525,3.382813,4.569495,9,97,11901
98_counts_clusters_a34405d3,2.191547,5.792474,1.799250,20.217224,0.784272,0.465956,4.838669,3.831804,6.003469,1.157570,79.623646,9.886305,1.222871,4.735837,3.083184,4.082930,11,98,15650


In [10]:
df_cell_meta_cluster_channel_avg

channels,cell_meta_cluster,CD3,CD4,CD8,CD14,CD20,CD31,CD45,CD68,CD163,CK17,Collagen1,Fibronectin,ECAD,HLADR,SMA,Vim,cell_meta_cluster_rename
0,1,9.920224,10.636219,9.059409,11.995957,6.404475,7.600641,19.981869,5.353948,7.567123,5.640777,17.392973,13.367731,5.794182,6.907827,9.516907,19.260004,1
1,2,9.960207,15.345797,6.050494,21.287334,5.233185,2.117434,17.084238,6.750041,13.714162,2.039449,20.468669,16.143988,4.065160,9.322342,6.876040,14.503086,2
2,3,20.953209,26.071361,8.986690,11.771607,8.442361,1.953716,27.605551,7.085774,8.561212,3.263422,14.460568,10.752086,4.982453,8.778626,5.584443,14.549717,3
3,4,14.599806,19.162653,6.738744,10.326302,31.657597,2.134133,36.219244,4.099054,9.283807,2.693036,10.250166,8.842964,4.352090,11.591215,2.726349,11.210159,4
4,5,2.607173,3.224016,1.587835,4.004721,1.092902,0.628333,3.484541,2.121840,2.685696,0.959712,8.164292,5.457338,3.288760,2.259438,3.133704,3.889296,5
5,6,3.951987,13.003755,3.170035,43.084493,1.789685,1.112180,10.437545,24.989896,19.458082,1.397150,15.664330,9.878738,2.170529,11.558954,5.503335,12.998675,6
6,7,5.573563,10.668688,3.377258,18.102451,2.561174,4.944619,9.995903,5.517863,9.383994,1.853947,18.188148,14.227881,3.399089,7.011984,9.899051,38.214906,7
7,8,27.069442,35.805355,5.529667,9.248749,16.994136,1.639795,36.993515,3.775317,7.440898,2.550704,10.308884,8.615445,4.747283,8.732910,2.593586,10.743067,8
8,9,15.020223,19.070327,6.534041,10.172453,22.049004,2.585827,30.746578,4.145985,8.132600,2.868514,13.845175,11.285477,6.531780,11.375026,3.932905,12.996394,9
9,10,6.325782,10.142026,4.140835,12.977858,2.843029,3.477757,10.121100,5.695717,7.105341,2.085989,21.844039,16.293455,4.120571,6.143693,13.951618,33.232461,10


In [11]:
# "table_cell_clustering_flowsom" is annotated by segmentation masks, so they can be visualised using napari-spatialdata
sdata_ark_analysis[ "table_cell_clustering_flowsom" ].uns[ "spatialdata_attrs" ]

#from napari_spatialdata import Interactive

#Interactive(sdata_ark_analysis)

{'region': ['label_whole_fov0', 'label_whole_fov1'],
 'region_key': 'fov_labels',
 'instance_key': 'cell_ID'}